In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('data/health.csv')
data = data.drop(columns = ['Unnamed: 0'])

In [7]:
data.columns

Index(['index', 'SNRLGSVC', 'SNRLGIMP', 'SNRLDCSN', 'SNRLFRND', 'YEAR',
       'CASEID', 'Tabacco_Flag', 'Alcohol_Flag', 'Marijunana_Flag',
       'Marijuana_Only_Flag', 'Cocaine_Flag', 'Crack_Flag', 'Herion_Flag',
       'Hallcinogens_Flag', 'Inhalants_Flag', 'Painkiller_Flag',
       'Tranquilizer_Flag', 'Stimulant Flag', 'CPN_Meth_Flag', 'Meth_Flag',
       'Sedatives Flag', 'Any_Drug_Flag', 'Any_Drug_Past_Year',
       'Any_Drug_Past_Month', 'Age_First_Use', 'Year_First_Use',
       'Drug_Not_Marijunana', 'Ever_Steal_50', 'Sell_Illegal_Drugs',
       'Ever_Booked_Arrested', 'Drug_or_Alc_Past_year', 'inpatient',
       'outpatient', 'needed_not_rcvd', 'rcvd_any_tmt',
       'suic_thought_pst_12month', 'psych_distress_month', 'psych_distress_yr',
       'suic_thought_pst_yr', 'suic_plan', 'suic_attempt', 'whodas_score',
       'predicted_ment_ill', 'svrl_days_depressed', 'lifetime_mde',
       'pst_yr_mde', 'CATAG6', 'HEALTH', 'IRSEX', 'IRMARIT', 'EDUCCAT2',
       'NEWRACE2', 'EMPST

In [12]:
suic = data[['suic_thought_pst_12month', 'suic_thought_pst_yr', 'suic_plan', 'suic_attempt']]

In [17]:
suic.suic_attempt.value_counts()

0.0    277812
1.0      2513
Name: suic_attempt, dtype: int64